# 第13章 接口、协议和抽象基类

- 鸭子类型：无需使用`isinstance`检查。
- 大鹅类型：Python>=2.6，使用`isinstance`检查是否符合抽象基类的要求。
- 静态类型：Python>=3.5，依靠PEP484实现的类型提示和外部类型检查工具。
- 静态鸭子类型：Python>=3.8，依靠PEP544实现的`typing.Protocol`类型提示和外部类型检查工具。

## 13.1 动态协议和静态协议

**定义：**

- 动态协议：Python大多数重要的动态协议由解释器支持。
- 静态协议：使用`typing.Protocol`子类显式定义。

**区别：**

- 对象可以只实现动态协议的一部分，如果想满足静态协议，则对象必须提供协议类中声明的每一个方法，即使程序用不到。
- 静态协议可以使用静态类型检查工具确认，动态协议则不能。

## 13.2 利用鸭子类型编程

### 13.2.1 猴子补丁

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

猴子补丁：在运行时动态修改模块、类或函数，以增加功能或修正bug。

In [4]:
def set_card(deck, position, card):
    deck._cards[position] = card

# 打猴子补丁，把它变成可变序列
FrenchDeck.__setitem__ = set_card

In [5]:
from random import shuffle

deck = FrenchDeck()

shuffle(deck)
deck[:5]

[Card(rank='5', suit='clubs'),
 Card(rank='A', suit='diamonds'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='clubs'),
 Card(rank='Q', suit='spades')]

以上代码，强调了猴子补丁，在运行时修改类或模块，而不改动源码，打补丁的代码与被打补丁的程序耦合十分紧密，而且往往要处理文档没有明确说明的私有属性。

### 13.2.2 防御性编程和“快速失败”

- 如果一个函数接收一系列项，在内部按照列表处理，可以立即利用参数构造一个列表。

In [6]:
def __init__(self, iterable):
    self._balls = list(iterable)

- 如果数据太多，需要就地修改数据，则应该使用`isinstance(x, abc.MutableSequence)`。

- 如果担心传入的是无穷生成器，则可以先使用`len()`获取参数的长度，遇到无效参数会立即抛出错误。

## 13.3 大鹅类型

大鹅类型的要求：
- 定义抽象基类的子类，明确表明在实现既有的接口。
- 运行时检查类型时，`isinstance`和`issubclass`的第二个参数要使用抽象基类，而不是具体类。

### 13.3.1 子类化一个抽象基类

In [7]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)

**改进方案：**
1. 新增`__setitem__`方法，支持洗牌操作。
2. 继承了`MutableSequence`类，还需新增`__delitem__`方法和`insert`方法。

### 13.3.2 定义并使用一个抽象基类

In [10]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):         
        """从可迭代对象中添加元素"""

    @abc.abstractmethod
    def pick(self):  # <3>
        """随机删除元素，再返回被删除的元素。
        
        如果实例为空，那么这个方法应该抛出LookupError
        """

    def loaded(self):
        """如果至少有一个元素，就返回True，否则返回False"""
        return bool(self.inspect())

    def inspect(self):
        """返回由容器中的当前元素构成的有序元组"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  
        return tuple(items)

### 13.3.3 子类化抽象基类Tombola

In [11]:
import random

class BingoCage(Tombola):

    def __init__(self, items):
        # 使用随机发生器
        self._randomizer = random.SystemRandom()
        self._items = []
        # 实现初始加载
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        # 使用随机发生器打乱序列
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

上述代码的`load`方法很耗时和`inspect`方法很笨拙，都可以覆盖调，以下代码进行了优化。

In [12]:
class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        # 取出一个随机位置上的球
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(self._balls)

### 13.3.4 抽象基类的虚拟子类

大鹅类型的重要特性：
- 使用`register`的装饰器，将一个类注册为抽象基类的虚拟子类。
- 注册的类变成抽象基类的虚拟子类，而且`issubclass`函数能够识别这种关系，但是注册的类不会从抽象基类中集成任何方法或属性。

In [13]:
from random import randrange

@Tombola.register
class TomboList(list):  

    def pick(self):
        if self:  
            # 获得随机的元素索引
            position = randrange(len(self))
            return self.pop(position)  
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  

    def loaded(self):
        return bool(self)  

    def inspect(self):
        return tuple(self)

In [14]:
Tombola.register(TomboList)

__main__.TomboList

In [15]:
issubclass(TomboList, Tombola)

True

In [16]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [17]:
# 类的继承关系
TomboList.__mro__

(__main__.TomboList, list, object)

## 13.4 静态协议

### 13.4.1 为double函数添加类型提示

In [18]:
from typing import TypeVar, Protocol

# 定义泛型
T = TypeVar('T')

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...  

RT = TypeVar('RT', bound=Repeatable)

def double(x: RT) -> RT:
    return x * 2

### 13.4.2 设计一个静态协议

**运行时协议检查的局限性：**`isinstance`或`issubclass`只检查有没有特定的方法，不检查方法的签名，更不会检查方法的类型注解。

In [19]:
from typing import runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...

In [21]:
from typing import Iterable

class SimplePicker:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:
        return self._items.pop()

In [23]:
# 类型是相容的
popper: RandomPicker = SimplePicker([1])
assert isinstance(popper, RandomPicker)

### 13.4.3 协议设计约定

- 使用朴素的名称命名协议，清楚表明概念。
- 使用SupportsX形式命名提供可调用方法的协议。
- 使用HasX形式命名有可读属性和可写属性，或者有读值方法和设值方法的协议。